In [1]:
from Data_utils import *

2024-05-23 15:12:29.997834: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 15:12:30.044966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 15:12:30.045013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 15:12:30.046152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 15:12:30.053013: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from vgg import VGG
class DetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(DetectionModel, self).__init__()
        # Load a ResNet model but remove the fully connected layer
        base_model = VGG('VGG16', 1, num_classes)
        self.features = nn.Sequential(*list(base_model.children())[:-1])  # Remove the last layer
        self.num_classes=num_classes

        # Output layer for class predictions
        self.classifier = nn.Linear(7*7*512, num_classes)  # 512 is the feature size of ResNet34 at the end

        # Output layers for bounding box predictions
        # 4 outputs per class: start_x, start_y, end_x, end_y
        self.bbox_regressor = nn.Linear(7*7*512, 4)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)

        # Class prediction
        classes = self.classifier(x)

        # Bounding box predictions
        bboxes = self.bbox_regressor(x)
        # Apply sigmoid to confine the outputs between 0 and 1
        bboxes = torch.sigmoid(bboxes)
        bboxes = bboxes.view(-1, 4)  # reshape to (batch_size, num_classes, 4)

        return classes, bboxes

In [3]:
# Train and Validation data loader
train_dataloader = DataLoader(MNISTSyntheticDataset(train=False), 
                        batch_size=32,  # Consider batch size based on your GPU memory
                        shuffle=True, 
                        num_workers=2)  # Increase this based on your system capabilities
val_dataloader = DataLoader(MNISTSyntheticDataset(train=False), 
                        batch_size=32,  # Consider batch size based on your GPU memory
                        shuffle=True, 
                        num_workers=2)  # Increase this based on your system capabilities



In [4]:
# CE + SmoothL1Loss
model = DetectionModel(num_classes=10).to('cuda')
loss_function = DetectionLoss(num_classes=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Assuming the train_loader and val_loader are properly defined
train_and_validate(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=20, visualize_after_train=True, log_dir='detection_logs_VGG/VGG16_CE_Smoothl1')
                   

Training Epoch 1/20:   0%|          | 0/313 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20] - Train classification Loss: 2.7035, Train iou Loss: 0.0268, Accuracy: 0.1774, IoU: 0.2707
Epoch [1/20] - Val classification Loss: 2.0599, Val iou Loss: 0.0067, Accuracy: 0.2789, IoU: 0.4497


Epoch [2/20] - Train classification Loss: 1.5499, Train iou Loss: 0.0049, Accuracy: 0.4245, IoU: 0.4747
Epoch [2/20] - Val classification Loss: 1.5511, Val iou Loss: 0.0117, Accuracy: 0.3963, IoU: 0.2107


Epoch [3/20] - Train classification Loss: 1.0958, Train iou Loss: 0.0033, Accuracy: 0.6133, IoU: 0.5089
Epoch [3/20] - Val classification Loss: 0.7831, Val iou Loss: 0.0028, Accuracy: 0.7385, IoU: 0.5146


Epoch [4/20] - Train classification Loss: 0.6891, Train iou Loss: 0.0025, Accuracy: 0.7725, IoU: 0.5375
Epoch [4/20] - Val classification Loss: 1.5633, Val iou Loss: 0.0224, Accuracy: 0.5588, IoU: 0.3049


...

Epoch [16/20] - Train classification Loss: 0.1176, Train iou Loss: 0.0009, Accuracy: 0.9646, IoU: 0.6542
Epoch [16/20] - Val classification Loss: 0.1453, Val iou Loss: 0.0013, Accuracy: 0.9558, IoU: 0.5974


Epoch [17/20] - Train classification Loss: 0.1050, Train iou Loss: 0.0010, Accuracy: 0.9684, IoU: 0.6482
Epoch [17/20] - Val classification Loss: 0.0768, Val iou Loss: 0.0009, Accuracy: 0.9752, IoU: 0.6534


Epoch [18/20] - Train classification Loss: 0.0993, Train iou Loss: 0.0010, Accuracy: 0.9689, IoU: 0.6463
Epoch [18/20] - Val classification Loss: 0.1068, Val iou Loss: 0.0012, Accuracy: 0.9674, IoU: 0.5982


Epoch [19/20] - Train classification Loss: 0.1075, Train iou Loss: 0.0010, Accuracy: 0.9676, IoU: 0.6506
Epoch [19/20] - Val classification Loss: 0.0887, Val iou Loss: 0.0008, Accuracy: 0.9737, IoU: 0.6730


Epoch [20/20] - Train classification Loss: 0.0994, Train iou Loss: 0.0009, Accuracy: 0.9710, IoU: 0.6577
Epoch [20/20] - Val classification Loss: 0.1189, Val iou Loss: 0.0008, Accuracy: 0.9618, IoU: 0.6718


In [5]:
# CE + IOU
model = DetectionModel(num_classes=10).to('cuda')
loss_function = CombinedLoss(num_classes=10,use_focal=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Assuming the train_loader and val_loader are properly defined
train_and_validate(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=20, visualize_after_train=True, log_dir='detection_logs_VGG/VGG16_CE_IOU')


Epoch [1/20] - Train classification Loss: 2.6215, Train iou Loss: 0.7226, Accuracy: 0.2045, IoU: 0.2814
Epoch [1/20] - Val classification Loss: 7.9336, Val iou Loss: 0.9340, Accuracy: 0.1354, IoU: 0.2153


Epoch [2/20] - Train classification Loss: 1.6524, Train iou Loss: 0.5244, Accuracy: 0.3905, IoU: 0.4936
Epoch [2/20] - Val classification Loss: 1.4179, Val iou Loss: 0.5450, Accuracy: 0.4558, IoU: 0.3987


Epoch [3/20] - Train classification Loss: 1.2753, Train iou Loss: 0.4450, Accuracy: 0.5311, IoU: 0.5656
Epoch [3/20] - Val classification Loss: 1.0328, Val iou Loss: 0.4195, Accuracy: 0.6367, IoU: 0.5828


Epoch [4/20] - Train classification Loss: 0.9028, Train iou Loss: 0.4231, Accuracy: 0.6838, IoU: 0.5877
Epoch [4/20] - Val classification Loss: 0.7441, Val iou Loss: 0.4569, Accuracy: 0.7440, IoU: 0.4815


Epoch [5/20] - Train classification Loss: 0.6530, Train iou Loss: 0.3956, Accuracy: 0.7862, IoU: 0.6130
Epoch [5/20] - Val classification Loss: 0.5670, Val iou Loss: 0.3514, Accuracy: 0.8189, IoU: 0.6627


...

Epoch [16/20] - Train classification Loss: 0.1024, Train iou Loss: 0.2417, Accuracy: 0.9702, IoU: 0.7690
Epoch [16/20] - Val classification Loss: 0.3050, Val iou Loss: 0.9735, Accuracy: 0.8963, IoU: 0.2157


Epoch [17/20] - Train classification Loss: 0.0875, Train iou Loss: 0.2354, Accuracy: 0.9748, IoU: 0.7765
Epoch [17/20] - Val classification Loss: 0.1241, Val iou Loss: 0.2167, Accuracy: 0.9625, IoU: 0.8007


Epoch [18/20] - Train classification Loss: 0.0978, Train iou Loss: 0.2319, Accuracy: 0.9726, IoU: 0.7810
Epoch [18/20] - Val classification Loss: 0.0653, Val iou Loss: 0.2327, Accuracy: 0.9785, IoU: 0.7852


Epoch [19/20] - Train classification Loss: 0.0850, Train iou Loss: 0.2222, Accuracy: 0.9753, IoU: 0.7867
Epoch [19/20] - Val classification Loss: 0.0862, Val iou Loss: 0.2369, Accuracy: 0.9775, IoU: 0.7713


Epoch [20/20] - Train classification Loss: 0.0854, Train iou Loss: 0.2256, Accuracy: 0.9745, IoU: 0.7908
Epoch [20/20] - Val classification Loss: 0.0969, Val iou Loss: 0.2143, Accuracy: 0.9709, IoU: 0.8054


In [4]:
# Focal loss + IOU
model = DetectionModel(num_classes=10).to('cuda')
loss_function = CombinedLoss(num_classes=10,use_focal=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Assuming the train_loader and val_loader are properly defined
train_and_validate(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=20, visualize_after_train=True, log_dir='detection_logs_VGG/VGG_focal_IOU')


Training Epoch 1/20:   0%|          | 0/313 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20] - Train classification Loss: 0.6582, Train iou Loss: 0.6247, Accuracy: 0.1122, IoU: 0.3687
Epoch [1/20] - Val classification Loss: 0.4808, Val iou Loss: 0.3951, Accuracy: 0.1237, IoU: 0.6161


Epoch [2/20] - Train classification Loss: 0.4900, Train iou Loss: 0.4253, Accuracy: 0.1327, IoU: 0.5755
Epoch [2/20] - Val classification Loss: 0.4623, Val iou Loss: 0.3673, Accuracy: 0.1258, IoU: 0.6416


Epoch [3/20] - Train classification Loss: 0.4585, Train iou Loss: 0.3399, Accuracy: 0.1513, IoU: 0.6819
Epoch [3/20] - Val classification Loss: 0.4612, Val iou Loss: 0.3057, Accuracy: 0.1652, IoU: 0.7083


Epoch [4/20] - Train classification Loss: 0.4281, Train iou Loss: 0.2955, Accuracy: 0.2002, IoU: 0.7190
Epoch [4/20] - Val classification Loss: 0.4331, Val iou Loss: 0.3275, Accuracy: 0.1579, IoU: 0.6477


...

Epoch [16/20] - Train classification Loss: 0.0347, Train iou Loss: 0.1909, Accuracy: 0.9134, IoU: 0.8402
Epoch [16/20] - Val classification Loss: 0.0355, Val iou Loss: 0.1836, Accuracy: 0.9066, IoU: 0.8556


Epoch [17/20] - Train classification Loss: 0.0350, Train iou Loss: 0.1866, Accuracy: 0.9133, IoU: 0.8394
Epoch [17/20] - Val classification Loss: 0.3099, Val iou Loss: 0.2628, Accuracy: 0.5560, IoU: 0.7398


Epoch [18/20] - Train classification Loss: 0.0334, Train iou Loss: 0.1912, Accuracy: 0.9164, IoU: 0.8390
Epoch [18/20] - Val classification Loss: 0.0930, Val iou Loss: 0.3190, Accuracy: 0.7619, IoU: 0.6927


Epoch [19/20] - Train classification Loss: 0.0292, Train iou Loss: 0.1810, Accuracy: 0.9251, IoU: 0.8490
Epoch [19/20] - Val classification Loss: 0.0266, Val iou Loss: 0.1622, Accuracy: 0.9339, IoU: 0.8762


Epoch [20/20] - Train classification Loss: 0.0351, Train iou Loss: 0.1834, Accuracy: 0.9161, IoU: 0.8426
Epoch [20/20] - Val classification Loss: 0.1428, Val iou Loss: 0.2123, Accuracy: 0.6614, IoU: 0.8143


In [5]:
!tar -zcvf detection_logs_VGG.tar.gz detection_logs_VGG

detection_logs_VGG/
detection_logs_VGG/VGG16_CE_Smoothl1/
detection_logs_VGG/VGG16_CE_Smoothl1/events.out.tfevents.1716424419.jupyter-ec23457.43834.0
detection_logs_VGG/VGG16_CE_Smoothl1/events.out.tfevents.1714918091.jupyter-ec23457.150879.0
detection_logs_VGG/VGG16_CE_Smoothl1/events.out.tfevents.1716422369.jupyter-ec23457.3761.0
detection_logs_VGG/VGG16_CE_Smoothl1/events.out.tfevents.1716422214.jupyter-ec23457.2683.1
detection_logs_VGG/VGG16_CE_Smoothl1/events.out.tfevents.1716422180.jupyter-ec23457.2683.0
detection_logs_VGG/VGG16_CE_Smoothl1/events.out.tfevents.1716423816.jupyter-ec23457.35145.0
detection_logs_VGG/VGG16_CE_Smoothl1/visualizations/
detection_logs_VGG/VGG16_CE_Smoothl1/visualizations/epoch_7_positive_visualizations.png
detection_logs_VGG/VGG16_CE_Smoothl1/visualizations/epoch_1_positive_visualizations.png
detection_logs_VGG/VGG16_CE_Smoothl1/visualizations/epoch_14_positive_visualizations.png
detection_logs_VGG/VGG16_CE_Smoothl1/visualizations/epoch_7_negative_visua